# Laboratorio 6: Análisis de tráfico de red
## Security Data Science
### Universidad del valle de Guatemala

Abner Ivan Garcia - 21285

In [1]:
from scapy.all import * 
import pandas as pd 
import numpy as np 
import binascii 
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
import matplotlib.pyplot as plt

# Preámbulo

In [ ]:
# Capture 25 paquetes de su red doméstica
captured_packets = sniff(count=25)

# Imprimiendo tipo de variable, longitud y contenido
print("Tipo de variable:", type(captured_packets))
print("Longitud de la variable:", len(captured_packets))
print("Contenido de la variable:", captured_packets)

# Imprimiendo el tipo de dato del primer paquete capturado
first_packet = captured_packets[0]
print("Tipo de dato del primer paquete capturado:", type(first_packet))

# Imprimiendo contenido de 5 paquetes
print("\nContenido de los primeros 5 paquetes capturados:")
for i in range(5):
    print("Paquete", i+1, ":", captured_packets[i].summary())

Tipo de variable: <class 'scapy.plist.PacketList'>
Longitud de la variable: 25
Contenido de la variable: <Sniffed: TCP:25 UDP:0 ICMP:0 Other:0>
Tipo de dato del primer paquete capturado: <class 'scapy.layers.l2.Ether'>

Contenido de los primeros 5 paquetes capturados:
Paquete 1 : Ether / IP / TCP 192.168.1.86:53005 > 20.42.65.90:https S
Paquete 2 : Ether / IP / TCP 192.168.1.86:53006 > 104.22.0.235:https S
Paquete 3 : Ether / IP / TCP 104.22.0.235:https > 192.168.1.86:53006 SA
Paquete 4 : Ether / IP / TCP 192.168.1.86:53006 > 104.22.0.235:https A
Paquete 5 : Ether / IP / TCP 192.168.1.86:53006 > 104.22.0.235:https PA / Raw


# Análisis estadístico

In [ ]:
paquete_pcap = rdpcap("analisis_paquetes.pcap") # obteniendo el archivo pcap 

df = pd.DataFrame() # creando un DataFrame vacío para almacenar los datos de los paquetes

# Iterando sobre los paquetes en paquete_pcap que se ha leído a partir del archivo pcap
for i, packet in enumerate(paquete_pcap):
    
    # Obtenemos el tiempo de captura del paquete 
    tiempo = packet.time
    
    # Verificamos si el paquete tiene la capa IP el cual contiene la dirección IP de origen y destino
    if IP in packet:
        # Si la capa IP existe, se accede a los campos 'src' y 'dst' donde se encuentran las direcciones IP de origen y destino
        src_address = packet[IP].src
        dst_address = packet[IP].dst
    else:
        src_address = None
        dst_address = None

    # Verificando si el paquete tiene la capa TCP o UDP
    if TCP in packet:
        # Si la capa TCP existe, se accede a los campos 'sport' y 'dport'
        src_port = packet[TCP].sport
        dst_port = packet[TCP].dport
    elif UDP in packet:
        # Si la capa UDP existe, se accede a los campos 'sport' y 'dport'
        src_port = packet[UDP].sport
        dst_port = packet[UDP].dport
    else:
        src_port = None # si no hay capa TCP o UDP, se asigna None
        dst_port = None # si no hay capa TCP o UDP, se asigna None
    
    # Verificando si el paquete tiene la capa Raw (payload)
    if Raw in packet:
        # Si la capa Raw existe, se accede al payload
        payload = len(packet[Raw].load)
    else:
        payload = 0 # si no hay capa Raw, se asigna 0 al payload

  # Añadiendo los campos al DataFrame
    df_temp = pd.DataFrame({'Time': [tiempo], 'Src Address': [src_address], 'Dst Address': [dst_address], 'Src Port': [src_port], 'Dst Port': [dst_port], 'Payload': [payload]})
    df = pd.concat([df, df_temp], ignore_index=True) # concatenando el DataFrame temporal al DataFrame principal

print(df.head()) # Imprimiendo las primeras filas del DataFrame 

                Time  Src Address  Dst Address  Src Port  Dst Port  Payload
0  1532199330.917674   10.1.10.53  84.54.22.33        53        53      905
1  1532199330.950107  84.54.22.33   10.1.10.53        53        53        0
2  1532199331.937777   10.1.10.53  84.54.22.33        53        53      919
3  1532199331.956635  84.54.22.33   10.1.10.53        53        53        0
4  1532199332.960504   10.1.10.53  84.54.22.33        53        53      956


# 5. Estadísticas
#### a. Muestre todas las IP origen

In [4]:
print("IPs de origen:")
print(df['Src Address'].unique())

IPs de origen:
['10.1.10.53' '84.54.22.33' '75.75.75.75']


#### b. Muestre todas las IP destino

In [5]:
print("\nIPs de destino:")
print(df['Dst Address'].unique())


IPs de destino:
['84.54.22.33' '10.1.10.53' '75.75.75.75']


#### c. ¿Cuál es la IP origen más frecuente?
i. ¿A qué IP destino se comunica con más frecuencia?

ii. ¿A que puerto destino se comunica? ¿Cuál es el propósito de este puerto?

iii. ¿Desde que puertos origen se comunica?

In [7]:
# ¿Cuál es la IP origen más frecuente?
src_ip_mas_frecuente = df['Src Address'].value_counts().idxmax() # obteniendo la IP origen más frecuente 
print("\nLa IP origen más frecuente es:", src_ip_mas_frecuente) # imprimiendo la IP origen más frecuente



La IP origen más frecuente es: 10.1.10.53


In [8]:
# i. ¿A qué IP destino se comunica con más frecuencia?
dst_ip_mas_frecuente = df[df['Src Address'] == src_ip_mas_frecuente]['Dst Address'].value_counts().idxmax() # obteniendo la IP destino más frecuente 
print("La IP destino con la que se comunica más frecuentemente es:", dst_ip_mas_frecuente) # imprimiendo la IP destino más frecuente

La IP destino con la que se comunica más frecuentemente es: 84.54.22.33


In [11]:
# ii. ¿A que puerto destino se comunica? ¿Cuál es el propósito de este puerto?
dst_port_mas_frecuente = df[(df['Src Address'] == src_ip_mas_frecuente) & (df['Dst Address'] == dst_ip_mas_frecuente)]['Dst Port'].value_counts().idxmax() # obteniendo el puerto destino más frecuente
print("El puerto destino con el que se comunica más frecuentemente es:", dst_port_mas_frecuente) # imprimiendo el puerto destino más frecuente


El puerto destino con el que se comunica más frecuentemente es: 53


#### ¿Cuál es el propósito de este puerto?
R// El puerto 53 es utilizado principalmente por el protocolo DNS, cuyo propósito es traducir nombres de dominio legibles por humanos en direcciones IP que las computadoras utilizan para comunicarse en la red.

In [10]:
# iii. ¿Desde que puertos origen se comunica?
src_ports = df[df['Src Address'] == src_ip_mas_frecuente]['Src Port'].unique() # obteniendo los puertos origen únicos
print("\nLos puertos origen desde los que se comunica son:", src_ports) # imprimiendo los puertos origen únicos


Los puertos origen desde los que se comunica son: [   53 15812 23903]
